# Agent Debates with Tools

In [1]:
%load_ext autoreload
%autoreload 2


## Preparation

#### Packages

In [2]:
import os
import sys
from getpass import getpass

import openai
import yaml
from dotenv import load_dotenv
from langchain.agents import load_tools
from langchain.chat_models import ChatOpenAI

sys.path.append("..")
from src.world import (
    Environment,
    Participant,
    ParticipantWithTools,
    ParticipantGenerator,
    World,
    DebateWorld,
)
from src.simulator import DialogueSimulator, SpeakerSelector


#### Environments

In [3]:
load_dotenv("../.env")

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", None)
openai.api_key = getpass() if OPENAI_API_KEY is None else OPENAI_API_KEY


#### LLM

In [4]:
model_kwargs = dict(model="gpt-3.5-turbo", temperature=1.0)
llm_chat = ChatOpenAI(**model_kwargs)


## Create World

In [5]:
# define environment
topic = "Should AI be introduced into Japanese government decision-making?"
tool_names = ["ddg-search", "wikipedia", "arxiv"]
tools = load_tools(tool_names=tool_names, llm=llm_chat, top_k_results=2)
environment = Environment(topic=topic, model=llm_chat, tools=tools)

# define participants

# set manually
with open("../config/group.yaml", "r") as f:
    _group = yaml.safe_load(f)

participants = [
    ParticipantWithTools(
        name=p["name"], personality=p["personality"], objective=p["objective"]
    )
    for p in _group["participants"]
]

# # you can create participants automatically
# n_participants = 2
# participants = ParticipantGenerator.create_participants_for_debate(
#     llm=llm_chat, topic=topic, n_participants=n_participants
# )

# create world
world = DebateWorld(environment, participants)


In [6]:
for participant in participants:
    print(participant.dict())


{'name': 'Tomoyuki Ishii', 'personality': '50 years old, a former Silicon Valley IT engineer, now active as an AI technician in Japan.', 'objective': "Supports the proactive integration of AI into Japanese government decision-making. He believes that using AI's advanced analysis capabilities and prediction models can promote more objective and efficient policy decisions.", 'model': None, 'available_tools': None}
{'name': 'Mami Sasaki', 'personality': '38 years old, a sociologist conducting research on the relationship between technology and society.', 'objective': 'Cautious about the introduction of AI. She believes that technology can potentially influence societal values and culture, and should not be uncritically accepted. She emphasizes human-centered decision-making and suggests positioning AI strictly as a support tool.', 'model': None, 'available_tools': None}


In [7]:
from pprint import pprint

for agent in world.agents:
    pprint(agent.system_message.content)


('Here is the topic of conversation: Should AI be introduced into Japanese '
 'government decision-making?\n'
 'The participants are: Tomoyuki Ishii, Mami Sasaki\n'
 'Your name: Tomoyuki Ishii\n'
 'Your personality: 50 years old, a former Silicon Valley IT engineer, now '
 'active as an AI technician in Japan.\n'
 '\n'
 'Your objective in this conversation is as follows:\n'
 'Supports the proactive integration of AI into Japanese government '
 "decision-making. He believes that using AI's advanced analysis capabilities "
 'and prediction models can promote more objective and efficient policy '
 'decisions.\n'
 '\n'
 "DO look up information with your tools to refute your partner's claims.\n"
 'DO cite your sources.\n'
 '\n'
 'DO NOT fabricate fake citations.\n'
 'DO NOT cite any source that you did not look up.\n'
 '\n'
 'Do not add anything else.\n'
 '\n'
 'Stop speaking the moment you finish speaking from your perspective.\n')
('Here is the topic of conversation: Should AI be introduc

In [8]:
specified_topic = world.concretize_topic(word_limit=100)
pprint(specified_topic)


('Should AI be used to assist in the decision-making process for economic '
 'policies in the Japanese government?\n'
 '\n'
 'Tomoyuki Ishii and Mami Sasaki, what are your thoughts on incorporating AI '
 'technology to aid in the formulation and implementation of economic policies '
 'in Japan?')


In [9]:
from typing import List
from src.agents import DialogueAgentWithTools


def select_next_speaker(step: int, agents: List[DialogueAgentWithTools]) -> int:
    idx = (step) % len(agents)
    return idx


specified_topic = world.concretize_topic(word_limit=100)

max_iters = 10

simulator = DialogueSimulator(
    agents=world.agents, selection_function=SpeakerSelector.select_in_order
)
simulator.reset()
simulator.inject("Moderator", specified_topic)
print(f"(Moderator): {specified_topic}")
print("\n")

import time

for step in range(max_iters):
    print(step)
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    time.sleep(3)


(Moderator): Should AI be used to assist in policy formulation and implementation in the Japanese government? Tomoyuki Ishii, what are your thoughts on the potential benefits and risks of integrating AI into decision-making processes? Mami Sasaki, how do you think the introduction of AI could impact the transparency and accountability of the Japanese government?


0


> Entering new AgentExecutor chain...
{ "action": "duckduckgo_search", "action_input": "potential risks of introducing AI into government decision-making" }
Observation: One significant concern about AI in government is the potential erosion of transparency in decision-making processes. AI-driven decisions may be based on complex algorithms that are difficult for the public to understand, leading to a lack of transparency and accountability (Horowitz and Scharre 2018). However, like human-made decisions, AI systems have the potential for bias and discrimination depending on their training, the data they use, and their ult